In [1]:
from pyspark import SparkContext
import pyspark

conf = pyspark.SparkConf().set("spark.driver.host", "localhost")
sc = SparkContext.getOrCreate()
sc.setLogLevel('OFF')

21/11/24 18:00:49 WARN Utils: Your hostname, Fans-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 2601:645:8303:3a10:0:0:0:a209 instead (on interface en0)
21/11/24 18:00:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/11/24 18:00:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/11/24 18:00:51 ERROR SparkContext: Error initializing SparkContext.
java.lang.AssertionError: assertion failed: Expected hostname or IPv6 IP enclosed in [] but got 2601:645:8303:3a10:0:0:0:a209
	at scala.Predef$.assert(Predef.scala:223)
	at org.apache.spark.util.Utils$.checkHost(Utils.scala:1079)
	at org.apache.spark.executor.Executor.<init>(Executor.scala:89)
	at org.apache.spar

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.AssertionError: assertion failed: Expected hostname or IPv6 IP enclosed in [] but got 2601:645:8303:3a10:0:0:0:a209
	at scala.Predef$.assert(Predef.scala:223)
	at org.apache.spark.util.Utils$.checkHost(Utils.scala:1079)
	at org.apache.spark.executor.Executor.<init>(Executor.scala:89)
	at org.apache.spark.scheduler.local.LocalEndpoint.<init>(LocalSchedulerBackend.scala:64)
	at org.apache.spark.scheduler.local.LocalSchedulerBackend.start(LocalSchedulerBackend.scala:132)
	at org.apache.spark.scheduler.TaskSchedulerImpl.start(TaskSchedulerImpl.scala:220)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:581)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at sun.reflect.De

In [2]:
supervisor_sf_pair = sc.textFile("../data/SF_business/supervisor_sf.tsv")\
                       .map(lambda x : x.split("\t"))\
                       .map(lambda x : (x[0], x[1]))

In [3]:
zip_biz_pair = sc.textFile("../data/SF_business/filtered_registered_business_sf.tsv")\
                 .map(lambda x : x.split("\t"))\
                 .map(lambda x : (x[0], x[1:]))

## 1. Return any five distinct  zip code in zip_biz_pair

In [5]:
zip_biz_pair.keys().distinct().take(5)

['94124', '94108', '94102', '94133', '94111']

In [6]:
zip_biz_pair.take(5)

[('94123', ['Tournahu George L', '3301 Broderick St', 'San Francisco', 'CA']),
 ('94124',
  ['Stephens Institute Inc', '2225 Jerrold Ave', 'San Francisco', 'CA']),
 ('94105',
  ['Stephens Institute Inc', '180 New Montgomery St', 'San Francisco', 'CA']),
 ('94108', ['Stephens Institute Inc', '540 Powell St', 'San Francisco', 'CA']),
 ('94107',
  ['Stephens Institute Inc', '460 Townsend St', 'San Francisco', 'CA'])]

## 2. Return any five distinct city names.

In [16]:
zip_biz_pair.values().map(lambda x: x[2]).distinct().take(5)

['San Francisco', 'Hayward', 'Redwood+city', 'Daly+city', 'Mexico']

## 3. Return five distinct (zip code, city name) pairs ordered by zip code, where zip code has 5 digits.

In [24]:
distinct_zip_city_pairs = zip_biz_pair.filter(lambda x: len(x[0])==5).mapValues(lambda x: x[2]).distinct().sortByKey()

In [25]:
distinct_zip_city_pairs.take(5)

[('10001', 'New+york'),
 ('10002', 'New+york'),
 ('10003', 'New+york'),
 ('10004', 'New+york'),
 ('10004', 'New York')]

## 4. Find the zip associated with the most city names
* which one is more efficient ??
    - zip_associated_with_multiple_cities_1  $vs$ zip_associated_with_multiple_cities_2

In [69]:
distinct_zip_city_pairs.mapValues(lambda x: 1).reduceByKey(lambda x,y: x+y).sortBy(lambda x: x[1],ascending=False).take(1)

[('94080', 40)]

In [70]:
distinct_zip_city_pairs.groupByKey().mapValues(len).sortBy(lambda x: x[1],ascending=False).take(1)

[('94080', 40)]

### Which one is more efficient? v1 or v2??

#### 

## 5. Create zip_city_count_names which includes (zip, (count, the list of names))

In [83]:
distinct_zip_city_pairs.take(5)

[('10001', 'New+york'),
 ('10002', 'New+york'),
 ('10003', 'New+york'),
 ('10004', 'New+york'),
 ('10004', 'New York')]

In [99]:
zip_city_count_names = distinct_zip_city_pairs.mapValues(lambda x: (1,[x])).reduceByKey(lambda x,y: (x[0]+y[0],sorted(x[1]+y[1]))).sortBy(lambda x: x[1][0],ascending=False)

In [100]:
zip_city_count_names.take(1)

[('94080',
  (40,
   ['Daly City',
    'S San Fran',
    'S San Francisco',
    'S+san+fran',
    'S+san+francisco',
    'S. San Francisco',
    'S.+san+francisco',
    'San Francisco',
    'San+bruno',
    'San+carlos',
    'San+francisco',
    'So San Francisco',
    'So+.+san+francisco',
    'So+san+francisco',
    'So.+san+francisco',
    'So.san+francisco',
    'Soith+san+francisco',
    'Souht+san+francisco',
    'Sout+san+francisco',
    'South San Francisco',
    'South San Fransico',
    'South Sanfrancisco',
    'South++san+francisco',
    'South+san+fracisco',
    'South+san+fran',
    'South+san+franc',
    'South+san+francisc',
    'South+san+francisco',
    'South+san+francisico',
    'South+san+francsico',
    'South+san+franisco',
    'South+san+franscisco',
    'South+san+fransisco',
    'South+san.francisco',
    'South+sanfrancisco',
    'South+sanfranciso',
    'South+sanfransico',
    'South+sf',
    'Ssf',
    'Suth+san+francisco']))]

## 6. convert zip_city_count_names to (zip, city_name)

In [102]:
zip_city_count_names.flatMapValues(lambda x: x[1]).take(5)

[('94080', 'Daly City'),
 ('94080', 'S San Fran'),
 ('94080', 'S San Francisco'),
 ('94080', 'S+san+fran'),
 ('94080', 'S+san+francisco')]

In [104]:
sc.stop()